In [31]:
import pandas as pd
from statistics import mean, median, stdev

#오류 메세지도 가린다
# import warnings
# warnings.filterwarnings('ignore')

In [32]:
selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0


In [33]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
dtype: object

In [34]:
dataCol = list(selloutData.columns)
dataCol

['REGIONID', 'PRODUCT', 'YEARWEEK', 'QTY']

In [35]:
# 1안

for i in range(0,len(dataCol)):
    try:
        selloutData[dataCol[i]] = selloutData[dataCol[i]].astype(float)
    except Exception as e:
        print(e)
selloutData['YEARWEEK'] = selloutData['YEARWEEK'].astype(str)
print(selloutData.dtypes)

could not convert string to float: 'A60'
could not convert string to float: 'PRODUCT4'
REGIONID     object
PRODUCT      object
YEARWEEK     object
QTY         float64
dtype: object


In [36]:
selloutData.describe()

,QTY
count,1.246580e+05
mean,8.949287e+03
std,4.294973e+04
min,-3.640000e+02
25%,3.200000e+01
50%,2.820000e+02
75%,2.223000e+03
max,1.663206e+06


In [37]:
# 이미 0인게 있다.
print(selloutData.loc[selloutData.QTY==0].shape)
print(selloutData.loc[selloutData.QTY<0].shape)

(10515, 4)
(323, 4)


In [38]:
# -인 값은 오류를 산출할 수 있으니 0으로 바꾼다.
selloutData.loc[selloutData["QTY"] < 0, "QTY"] = 0

In [39]:
#전체 갯수
print(selloutData.loc[selloutData.QTY>=0].shape)

(124658, 4)


In [40]:
# 몇개인지 확인, 기존에 0개였던 것과 0으로 바꾼 음수가 합쳐졌다
print(selloutData.loc[selloutData.QTY==0].shape)

# 0보다 큰건 몇개인지 확인
print(selloutData.loc[selloutData.QTY>0].shape)

(10838, 4)
(113820, 4)


In [41]:
#칼럼 조회 : 인덱스 전체 조회 후 칼럼명
REGIONID_DATA = selloutData.loc[:,["REGIONID"]]
PRODUCT_DATA = selloutData.loc[:,["PRODUCT"]]
YEARWEEK_DATA = selloutData.loc[:,["YEARWEEK"]]
QTY_DATA = selloutData.loc[:,["QTY"]]

In [42]:
# rolling을 이용하면 이동평균을 구해준다
# min_periods : 계산할 값의 최소 개수
# center=True : 현재 값을 중심으로 연산할지
selloutData["MA"] = round(selloutData.QTY.rolling(window=13, min_periods=1 ,center=True).mean(),3)
selloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,MA
0,A60,PRODUCT4,201402.0,71.0,4004.571
1,A60,PRODUCT59,201402.0,22275.0,3504.875
2,A60,PRODUCT34,201402.0,4463.0,3126.667
3,A60,PRODUCT47,201402.0,0.0,2847.600
4,A60,PRODUCT56,201402.0,23.0,2588.727
...,...,...,...,...,...
124653,A10,PRODUCT60,201630.0,824.0,25945.000
124654,A10,PRODUCT56,201630.0,275.0,28238.300
124655,A10,PRODUCT61,201630.0,0.0,31350.222
124656,A10,PRODUCT12,201630.0,15021.0,6391.000


In [43]:
selloutData["CR"] = round(selloutData.MA.rolling(window=5, min_periods=1 ,center=True).std(),3)
selloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,MA,CR
0,A60,PRODUCT4,201402.0,71.0,4004.571,440.351
1,A60,PRODUCT59,201402.0,22275.0,3504.875,500.993
2,A60,PRODUCT34,201402.0,4463.0,3126.667,557.327
3,A60,PRODUCT47,201402.0,0.0,2847.600,367.726
4,A60,PRODUCT56,201402.0,23.0,2588.727,290.077
...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630.0,824.0,25945.000,3600.100
124654,A10,PRODUCT56,201630.0,275.0,28238.300,9775.106
124655,A10,PRODUCT61,201630.0,0.0,31350.222,13426.383
124656,A10,PRODUCT12,201630.0,15021.0,6391.000,14813.921


In [44]:
# UPPER_BAND만들기
selloutData["UPPER_BAND"] = selloutData.CR

for i in range(0,len(selloutData)):
    selloutData["UPPER_BAND"][i] = selloutData.CR[i] + selloutData.QTY[i]
selloutData

C:\Users\caose\AppData\Local\Temp\ipykernel_5904\2317182594.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selloutData["UPPER_BAND"][i] = selloutData.CR[i] + selloutData.QTY[i]


,REGIONID,PRODUCT,YEARWEEK,QTY,MA,CR,UPPER_BAND
0,A60,PRODUCT4,201402.0,71.0,4004.571,440.351,511.351
1,A60,PRODUCT59,201402.0,22275.0,3504.875,500.993,22775.993
2,A60,PRODUCT34,201402.0,4463.0,3126.667,557.327,5020.327
3,A60,PRODUCT47,201402.0,0.0,2847.600,367.726,367.726
4,A60,PRODUCT56,201402.0,23.0,2588.727,290.077,313.077
...,...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630.0,824.0,25945.000,3600.100,4424.100
124654,A10,PRODUCT56,201630.0,275.0,28238.300,9775.106,10050.106
124655,A10,PRODUCT61,201630.0,0.0,31350.222,13426.383,13426.383
124656,A10,PRODUCT12,201630.0,15021.0,6391.000,14813.921,29834.921


In [ ]:
# LOWER_BAND
selloutData["LOWER_BAND"] = selloutData.CR

for i in range(0,len(selloutData)):
    selloutData["LOWER_BAND"][i] = selloutData.QTY[i] - selloutData.CR[i]
selloutData

C:\Users\caose\AppData\Local\Temp\ipykernel_5904\218226510.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selloutData["LOWER_BAND"][i] = selloutData.QTY[i] - selloutData.CR[i]


In [ ]:
selloutData.dtypes

In [ ]:
for i in range(0,len(selloutData)):
    if (selloutData["LOWER_BAND"][i] < 0) : 
        selloutData["LOWER_BAND"][i] = 0
selloutData

In [ ]:
selloutData["DATA_CLEANSING"] = 0

In [ ]:
for i in range(0,len(selloutData)):
    if(selloutData.QTY[i]>selloutData.UPPER_BAND[i]):
        selloutData["DATA_CLEANSING"][i] = selloutData.UPPER_BAND[i]
    elif(selloutData.QTY[i]<selloutData.LOWER_BAND[i]):        
        selloutData["DATA_CLEANSING"][i] = selloutData.LOWER_BAND[i]
    else :
        selloutData["DATA_CLEANSING"][i] = selloutData.QTY[i]
selloutData

In [ ]:
selloutData["SMOOTHING_1"] = round(selloutData.MA.rolling(window=5, min_periods=1 ,center=True).mean(),3)
selloutData

In [ ]:
selloutData["SMOOTHING_2"] = round(selloutData.DATA_CLEANSING.rolling(window=5, min_periods=1 ,center=True).mean(),3)
selloutData

In [ ]:
selloutData["FORECAST"] = 0
for i in range(0,len(selloutData)):
    selloutData["FORECAST"][i] = selloutData.QTY[i] - selloutData.SMOOTHING_2[i]
    if selloutData["FORECAST"][i] < 0 :
        selloutData["FORECAST"][i] = 0 
selloutData

In [ ]:
selloutData["FORECAST_NOTSAFE"] = 0
for i in range(0,len(selloutData)):
    selloutData["FORECAST_NOTSAFE"][i] = selloutData.DATA_CLEANSING[i] - selloutData.SMOOTHING_2[i]
    if selloutData["FORECAST_NOTSAFE"][i] < 0 :
        selloutData["FORECAST_NOTSAFE"][i] = 0 
selloutData